## Convolutional Neural Network
   #### Computer Vision Winter Semester 2020/2021 by Clemens Spielvogel
    
In the following code example you will see how the keras ImageDataGenerator can be used for data augmentation. Data augmentation increases the generalization of your model, can be used to handle imbalanced data sets and helps building good models on small amounts of data.

In [15]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "" # If you want to use TF CPU version while having the GPU version installed

import tensorflow as tf
from keras import Sequential, layers, regularizers
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import plotly.offline as ply
import plotly.graph_objs as graphs
import math
import random
import matplotlib.pyplot as plt

In [16]:
# Seeding random number generators to obtain reproducible results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value) # Resets itself on every use!
tf.random.set_seed(seed_value) # tf.set_random_seed(seed_value) on older TF versions

In [17]:
def load_data(file_path):
    # Load CSV file to pandas data frame
    df = pd.read_csv(file_path, header=0)

    # Split data frame columns into features and labels
    y = df["label"].values
    X_flat = df.drop("label", axis=1).values
    X = [np.resize(array, (28, 28)) for array in X_flat] # Resize flat vector to 28x28
    X = np.array([np.reshape(sample, (sample.shape[0], sample.shape[1], 1)) for sample in X]) # Add color channel
    
    return X, keras.utils.to_categorical(y) # Labels are returned as one-hot encoded vectors

In [18]:
def plot_train_val_performances(performances, metric="Accuracy", show=True):
    """Plot training vs. testing accuracy over all epochs. performances is a dictionary mapping epoch numbers
       as integers to lists containing training and validation performance (e.g. accuracy).
       metric is a string indicating the used performance metric."""
    x = list(performances.keys())     # Number of epochs
    y_train = [i[0] for i in performances.values()]
    y_val = [i[1] for i in performances.values()]

    trace_train = graphs.Scatter(x=x, y=y_train, name="Training", mode="lines+markers",
                                 line=dict(width=4),
                                 marker=dict(symbol="circle",
                                             size=10))
    trace_val = graphs.Scatter(x=x, y=y_val, name="Validation", mode="lines+markers",
                                line=dict(width=4),
                                marker=dict(symbol="circle",
                                            size=10))

    layout = graphs.Layout(title="Training vs. Validation {}".format(metric),
                           xaxis={"title": "Epoch"},
                           yaxis={"title": metric})

    fig = graphs.Figure(data=[trace_train, trace_val], layout=layout)
    ply.plot(fig, filename="plotly_train_val_{}.html".format(metric), auto_open=show)
    print("Plot saved as plotly_train_val_{}.html".format(metric))

In [19]:
def create_2DCNN_model(input_shape):
    """Build architecture of the model"""
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=input_shape,
                            activation="relu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(3, 3)))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.Conv2D(64, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation="selu", padding="same"))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), padding="same"))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation="selu"))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(32, activation="selu"))
    model.add(layers.Dense(10, activation="softmax"))

    # Create model
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

In [20]:
# Specify data location
data_dir = r"Data\mnist.csv"

# Set hyperparameters
num_epochs = 50
batch_size = 64
dims = (28, 28, 1)

In [21]:
# Load data
X, y = load_data(data_dir)

# Determine split sizes
train_size = math.floor(0.02 * len(y)) # ONLY 2 % of the data set is used for training to demonstrate the augmentation benefit
val_size = math.floor(0.475 * len(y))
test_size = math.floor(0.475 * len(y))

# Create splitted sets
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

In [22]:
# Create CNN model
model = create_2DCNN_model(dims)

In [23]:
# Data augmentation using Keras' built-in data generator
train_generator = ImageDataGenerator(rotation_range=10,
                                     shear_range = 0.1,
                                     zoom_range = 0.1)

In [24]:
# Configure stopping criterion via early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=4)

In [25]:
# Training model while dynamically retrieving the augmented data from the generator
train_summary = model.fit_generator(generator=train_generator.flow(X_train, y_train, batch_size=32),
                                    validation_data=(X_val, y_val),
                                    epochs=num_epochs,
                                    callbacks=[callback],
                                    verbose=1)

Epoch 1/50
7/7 [==============================] - 2s 289ms/step - loss: 7.8775 - accuracy: 0.0900 - val_loss: 2.8142 - val_accuracy: 0.1573
Epoch 2/50
7/7 [==============================] - 1s 110ms/step - loss: 2.8924 - accuracy: 0.1400 - val_loss: 1.8639 - val_accuracy: 0.3665
Epoch 3/50
7/7 [==============================] - 1s 106ms/step - loss: 2.1228 - accuracy: 0.3350 - val_loss: 2.0160 - val_accuracy: 0.3531
Epoch 4/50
7/7 [==============================] - 1s 106ms/step - loss: 1.8576 - accuracy: 0.4250 - val_loss: 1.5311 - val_accuracy: 0.4560
Epoch 5/50
7/7 [==============================] - 1s 107ms/step - loss: 1.4549 - accuracy: 0.5300 - val_loss: 1.1711 - val_accuracy: 0.6116
Epoch 6/50
7/7 [==============================] - 1s 108ms/step - loss: 1.0610 - accuracy: 0.6200 - val_loss: 0.8943 - val_accuracy: 0.7173
Epoch 7/50
7/7 [==============================] - 1s 106ms/step - loss: 0.6975 - accuracy: 0.7600 - val_loss: 0.7550 - val_accuracy: 0.7762
Epoch 8/50
7/7 [====

In [26]:
# Evaluate fitted model using test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("\nTest ACC:", round(test_acc, 3))

5050/5050 [==============================] - 1s 139us/step

Test ACC: 0.915


In [27]:
# Get epochwise performances
train_acc = train_summary.history["accuracy"]
val_acc = train_summary.history["val_accuracy"]

# Format and store performances per epoch for plotting
accs = {epoch: [round(performance[0], 2), round(performance[1], 2)]
        for epoch, performance in enumerate(zip(train_acc, val_acc))}

# Save model
model.save_weights("model_weights.h5")

# Plot training and validation performance over epochs
plot_train_val_performances(accs, "Accuracy")

Plot saved as plotly_train_val_Accuracy.html
